In [1]:
!pip install llama-index llama-index-llms-openai gradio pandas

In [2]:
import os
import pandas as pd
import gradio as gr
from datetime import datetime

from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import FunctionAgent

In [3]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "여러분들의 키 값"

In [4]:
# 확장된 샘플 데이터
products_data = [
    {"상품ID": "P001", "상품명": "프리미엄 강아지 간식 세트", "카테고리": "간식", "재고수량": 36, "가격": 32417, "제조사": "펫스토리", "태그": "간식,강아지,프리미엄", "판매량": 890},
    {"상품ID": "P002", "상품명": "고양이 캣타워 XL", "카테고리": "가구", "재고수량": 91, "가격": 47747, "제조사": "캣플레이", "태그": "고양이,캣타워,대형", "판매량": 245},
    {"상품ID": "P003", "상품명": "자동 급식기", "카테고리": "전자제품", "재고수량": 10, "가격": 89900, "제조사": "스마트펫", "태그": "자동급식기,스마트", "판매량": 567},
    {"상품ID": "P004", "상품명": "이동가방 대형", "카테고리": "외출용품", "재고수량": 99, "가격": 91542, "제조사": "펫캐리", "태그": "이동가방,여행,대형견", "판매량": 120},
    {"상품ID": "P005", "상품명": "펫 샴푸", "카테고리": "목욕용품", "재고수량": 49, "가격": 41276, "제조사": "펫케어", "태그": "샴푸,목욕,미용", "판매량": 678},
    {"상품ID": "P006", "상품명": "LED 목줄", "카테고리": "외출용품", "재고수량": 75, "가격": 18900, "제조사": "세이프펫", "태그": "목줄,LED,야간산책", "판매량": 445},
    {"상품ID": "P007", "상품명": "칫솔 치약 세트", "카테고리": "구강관리", "재고수량": 120, "가격": 15500, "제조사": "펫덴탈", "태그": "칫솔,치약,구강관리", "판매량": 234},
    {"상품ID": "P008", "상품명": "스크래처", "카테고리": "가구", "재고수량": 45, "가격": 35900, "제조사": "캣홈", "태그": "스크래처,고양이,발톱", "판매량": 189}
]

customers_data = [
    {"고객ID": "C001", "고객명": "박예준", "전화번호": "010-1234-5678", "이메일": "park@example.com", "가입일자": "2020-08-10", "등급": "VIP", "포인트": 5010, "총구매액": 892540, "구매횟수": 12},
    {"고객ID": "C002", "고객명": "김미숙", "전화번호": "010-2345-6789", "이메일": "kim@example.com", "가입일자": "2021-01-23", "등급": "VIP", "포인트": 5771, "총구매액": 456780, "구매횟수": 5},
    {"고객ID": "C003", "고객명": "이준호", "전화번호": "010-3456-7890", "이메일": "lee@example.com", "가입일자": "2022-07-10", "등급": "일반", "포인트": 2119, "총구매액": 235680, "구매횟수": 3},
]

orders_data = [
    {"주문ID": "O0001", "고객ID": "C001", "상품ID": "P001", "수량": 2, "주문일자": "2025-03-25", "결제상태": "결제완료", "배송ID": "D0001", "최종결제액": 53346},
    {"주문ID": "O0002", "고객ID": "C001", "상품ID": "P006", "수량": 1, "주문일자": "2025-03-20", "결제상태": "결제완료", "배송ID": "D0002", "최종결제액": 18900},
    {"주문ID": "O0003", "고객ID": "C001", "상품ID": "P003", "수량": 1, "주문일자": "2025-02-28", "결제상태": "결제완료", "배송ID": "D0003", "최종결제액": 89900},
    {"주문ID": "O0004", "고객ID": "C001", "상품ID": "P002", "수량": 1, "주문일자": "2025-01-15", "결제상태": "취소", "배송ID": "D0004", "최종결제액": 47747},
    {"주문ID": "O0005", "고객ID": "C002", "상품ID": "P005", "수량": 2, "주문일자": "2025-03-18", "결제상태": "결제완료", "배송ID": "D0005", "최종결제액": 82552},
    {"주문ID": "O0006", "고객ID": "C003", "상품ID": "P007", "수량": 3, "주문일자": "2025-03-10", "결제상태": "결제완료", "배송ID": "D0006", "최종결제액": 46500},
]

deliveries_data = [
    {"배송ID": "D0001", "배송상태": "배송중", "출고일자": "2025-03-26", "도착예정일": "2025-03-28", "배송사": "CJ대한통운", "송장번호": "123456789"},
    {"배송ID": "D0002", "배송상태": "배송완료", "출고일자": "2025-03-21", "도착예정일": "2025-03-23", "배송사": "한진택배", "송장번호": "987654321"},
    {"배송ID": "D0003", "배송상태": "배송완료", "출고일자": "2025-03-01", "도착예정일": "2025-03-03", "배송사": "우체국택배", "송장번호": "555666777"},
    {"배송ID": "D0004", "배송상태": "취소", "출고일자": "-", "도착예정일": "-", "배송사": "-", "송장번호": "-"},
    {"배송ID": "D0005", "배송상태": "상품준비중", "출고일자": "-", "도착예정일": "2025-03-30", "배송사": "롯데택배", "송장번호": "-"},
    {"배송ID": "D0006", "배송상태": "배송중", "출고일자": "2025-03-11", "도착예정일": "2025-03-13", "배송사": "CJ대한통운", "송장번호": "111222333"},
]

cart_data = [
    {"장바구니ID": "CART001", "고객ID": "C001", "상품ID": "P008", "수량": 1, "추가일자": "2025-03-26"},
    {"장바구니ID": "CART002", "고객ID": "C001", "상품ID": "P007", "수량": 2, "추가일자": "2025-03-25"},
    {"장바구니ID": "CART003", "고객ID": "C002", "상품ID": "P004", "수량": 1, "추가일자": "2025-03-24"},
]

# C001 리뷰 추가
reviews_data = [
    {"리뷰ID": "R001", "고객ID": "C001", "상품ID": "P001", "평점": 5, "리뷰내용": "강아지가 정말 좋아해요. 재구매 의사 있습니다!", "작성일자": "2025-03-27"},
    {"리뷰ID": "R002", "고객ID": "C001", "상품ID": "P006", "평점": 4, "리뷰내용": "LED가 밝아서 밤 산책때 안전해요. 배터리도 오래갑니다.", "작성일자": "2025-03-25"},
    {"리뷰ID": "R003", "고객ID": "C001", "상품ID": "P003", "평점": 5, "리뷰내용": "자동급식기 정말 편리해요. 외출할 때 걱정이 덜어요.", "작성일자": "2025-03-10"},
    {"리뷰ID": "R004", "고객ID": "C002", "상품ID": "P005", "평점": 3, "리뷰내용": "향은 좋은데 거품이 잘 안나요", "작성일자": "2025-03-20"},
    {"리뷰ID": "R005", "고객ID": "C003", "상품ID": "P007", "평점": 5, "리뷰내용": "치약 향도 좋고 강아지가 양치질 싫어하지 않아요", "작성일자": "2025-03-15"},
]

point_history_data = [
    {"포인트ID": "PT001", "고객ID": "C001", "구분": "적립", "포인트": 533, "내용": "O0001 구매 적립", "일자": "2025-03-25"},
    {"포인트ID": "PT002", "고객ID": "C001", "구분": "사용", "포인트": -2000, "내용": "O0003 구매시 사용", "일자": "2025-02-28"},
    {"포인트ID": "PT003", "고객ID": "C001", "구분": "적립", "포인트": 1000, "내용": "리뷰 작성 보너스", "일자": "2025-03-27"},
    {"포인트ID": "PT004", "고객ID": "C002", "구분": "적립", "포인트": 825, "내용": "O0005 구매 적립", "일자": "2025-03-18"},
]

promotions_data = [
    {"프로모션ID": "E001", "제목": "봄맞이 펫용품 대전", "내용": "전 상품 10% 할인 + 5만원 이상 구매시 사료샘플 증정", "시작일": "2025-03-01", "종료일": "2025-03-31", "대상상품": "전체"},
    {"프로모션ID": "E002", "제목": "신규회원 웰컴 혜택", "내용": "첫 구매시 15% 할인쿠폰 + 적립금 2배", "시작일": "2025-01-01", "종료일": "2025-12-31", "대상상품": "전체"},
    {"프로모션ID": "E003", "제목": "고양이 용품 특가전", "내용": "고양이 관련 상품 20% 할인", "시작일": "2025-03-15", "종료일": "2025-04-15", "대상상품": "고양이"},
]

In [5]:
# 데이터프레임 생성
products_df = pd.DataFrame(products_data)
customers_df = pd.DataFrame(customers_data)
orders_df = pd.DataFrame(orders_data)
deliveries_df = pd.DataFrame(deliveries_data)
cart_df = pd.DataFrame(cart_data)
reviews_df = pd.DataFrame(reviews_data)
point_history_df = pd.DataFrame(point_history_data)
promotions_df = pd.DataFrame(promotions_data)

In [6]:
def get_customer_profile(customer_id: str) -> str:
    """고객 ID로 고객 프로필 정보를 조회합니다."""
    customer = customers_df[customers_df['고객ID'] == customer_id]
    if customer.empty:
        return f"고객 ID {customer_id}를 찾을 수 없습니다."

    c = customer.iloc[0]
    result = []
    result.append(f"고객ID: {c['고객ID']}")
    result.append(f"고객명: {c['고객명']}")
    result.append(f"전화번호: {c['전화번호']}")
    result.append(f"이메일: {c['이메일']}")
    result.append(f"가입일자: {c['가입일자']}")
    result.append(f"고객등급: {c['등급']}")
    result.append(f"보유포인트: {c['포인트']:,}점")
    result.append(f"총구매액: {c['총구매액']:,}원")
    result.append(f"구매횟수: {c['구매횟수']}회")

    return "\n".join(result)

def search_products(keyword: str = None, category: str = None,
                   price_min: int = None, price_max: int = None) -> str:
    """키워드로 상품을 검색합니다."""
    df = products_df.copy()

    if keyword:
        mask = (
            df['상품명'].str.contains(keyword, case=False, na=False) |
            df['태그'].str.contains(keyword, case=False, na=False) |
            df['제조사'].str.contains(keyword, case=False, na=False)
        )
        df = df[mask]

    if category:
        df = df[df['카테고리'].str.contains(category, case=False, na=False)]

    if price_min:
        df = df[df['가격'] >= price_min]

    if price_max:
        df = df[df['가격'] <= price_max]

    if df.empty:
        return "조건에 맞는 상품이 없습니다."

    result = [f"검색 결과: {len(df)}개 상품\n"]

    for _, p in df.iterrows():
        result.append(f"상품ID: {p['상품ID']}")
        result.append(f"상품명: {p['상품명']}")
        result.append(f"가격: {p['가격']:,}원")
        result.append(f"재고: {p['재고수량']}개")
        result.append(f"카테고리: {p['카테고리']}")
        result.append("-------------------")

    return "\n".join(result)

def get_customer_orders(customer_id: str, start_date: str = None, end_date: str = None) -> str:
    """고객의 주문 내역을 조회합니다."""
    orders = orders_df[orders_df['고객ID'] == customer_id].copy()

    if orders.empty:
        return f"고객 ID {customer_id}의 주문 내역이 없습니다."

    if start_date:
        orders = orders[orders['주문일자'] >= start_date]
    if end_date:
        orders = orders[orders['주문일자'] <= end_date]

    if orders.empty:
        return "해당 기간의 주문 내역이 없습니다."

    result = [f"주문 내역 ({len(orders)}건):\n"]

    for _, order in orders.iterrows():
        product = products_df[products_df['상품ID'] == order['상품ID']]
        product_name = product.iloc[0]['상품명'] if not product.empty else "상품정보없음"

        result.append(f"주문ID: {order['주문ID']}")
        result.append(f"주문일자: {order['주문일자']}")
        result.append(f"상품: {product_name}")
        result.append(f"수량: {order['수량']}개")
        result.append(f"결제금액: {order['최종결제액']:,}원")
        result.append(f"결제상태: {order['결제상태']}")
        result.append("-------------------")

    return "\n".join(result)

def get_delivery_status(customer_id: str = None, order_id: str = None) -> str:
    """배송 상태를 확인합니다."""
    if order_id:
        order = orders_df[orders_df['주문ID'] == order_id]
        if order.empty:
            return f"주문번호 {order_id}를 찾을 수 없습니다."

        delivery_id = order.iloc[0]['배송ID']
        delivery = deliveries_df[deliveries_df['배송ID'] == delivery_id]

        if delivery.empty:
            return "배송 정보를 찾을 수 없습니다."

        d = delivery.iloc[0]
        product = products_df[products_df['상품ID'] == order.iloc[0]['상품ID']]
        product_name = product.iloc[0]['상품명'] if not product.empty else "상품정보없음"

        result = []
        result.append(f"주문번호: {order_id}")
        result.append(f"상품: {product_name}")
        result.append(f"배송상태: {d['배송상태']}")
        if d['배송상태'] != '취소':
            result.append(f"배송사: {d['배송사']}")
            result.append(f"송장번호: {d['송장번호']}")
            result.append(f"출고일자: {d['출고일자']}")
            result.append(f"도착예정일: {d['도착예정일']}")

        return "\n".join(result)

    elif customer_id:
        orders = orders_df[orders_df['고객ID'] == customer_id]
        if orders.empty:
            return f"고객 ID {customer_id}의 주문이 없습니다."

        result = ["배송 현황:\n"]

        for _, order in orders.iterrows():
            if order['결제상태'] == '결제완료':
                delivery = deliveries_df[deliveries_df['배송ID'] == order['배송ID']]
                if not delivery.empty:
                    d = delivery.iloc[0]
                    product = products_df[products_df['상품ID'] == order['상품ID']]
                    product_name = product.iloc[0]['상품명'] if not product.empty else "상품정보없음"

                    result.append(f"주문번호: {order['주문ID']}")
                    result.append(f"상품: {product_name}")
                    result.append(f"배송상태: {d['배송상태']}")
                    result.append("-------------------")

        return "\n".join(result)

    return "고객ID나 주문ID를 입력해주세요."

def search_reviews(keyword: str = None, product_name: str = None,
                  customer_id: str = None, rating: int = None) -> str:
    """리뷰를 검색합니다."""
    df = reviews_df.copy()

    if customer_id:
        df = df[df['고객ID'] == customer_id]

    if product_name:
        matching_products = products_df[
            products_df['상품명'].str.contains(product_name, case=False, na=False)
        ]['상품ID'].tolist()
        if matching_products:
            df = df[df['상품ID'].isin(matching_products)]

    if keyword:
        df = df[df['리뷰내용'].str.contains(keyword, case=False, na=False)]

    if rating:
        df = df[df['평점'] == rating]

    if df.empty:
        return "조건에 맞는 리뷰가 없습니다."

    result = [f"리뷰 검색 결과 ({len(df)}개):\n"]

    for _, review in df.iterrows():
        product = products_df[products_df['상품ID'] == review['상품ID']]
        product_name = product.iloc[0]['상품명'] if not product.empty else "상품정보없음"

        customer = customers_df[customers_df['고객ID'] == review['고객ID']]
        customer_name = customer.iloc[0]['고객명'] if not customer.empty else "고객정보없음"

        result.append(f"상품: {product_name}")
        result.append(f"평점: {'★' * review['평점']}")
        result.append(f"리뷰: {review['리뷰내용']}")
        result.append(f"작성자: {customer_name}")
        result.append(f"작성일: {review['작성일자']}")
        result.append("-------------------")

    return "\n".join(result)

def get_customer_cart(customer_id: str) -> str:
    """고객의 장바구니를 조회합니다."""
    cart_items = cart_df[cart_df['고객ID'] == customer_id]

    if cart_items.empty:
        return f"고객 ID {customer_id}의 장바구니가 비어있습니다."

    result = [f"장바구니 ({len(cart_items)}개 상품):\n"]
    total = 0

    for _, item in cart_items.iterrows():
        product = products_df[products_df['상품ID'] == item['상품ID']]
        if not product.empty:
            p = product.iloc[0]
            subtotal = p['가격'] * item['수량']
            total += subtotal

            result.append(f"상품: {p['상품명']}")
            result.append(f"수량: {item['수량']}개")
            result.append(f"단가: {p['가격']:,}원")
            result.append(f"소계: {subtotal:,}원")
            result.append("-------------------")

    result.append(f"\n총액: {total:,}원")
    result.append(f"배송비: {0 if total >= 50000 else 3000:,}원")
    result.append(f"결제예상액: {total + (0 if total >= 50000 else 3000):,}원")

    return "\n".join(result)

def get_point_history(customer_id: str) -> str:
    """포인트 내역을 조회합니다."""
    customer = customers_df[customers_df['고객ID'] == customer_id]
    if customer.empty:
        return f"고객 ID {customer_id}를 찾을 수 없습니다."

    current_points = customer.iloc[0]['포인트']
    history = point_history_df[point_history_df['고객ID'] == customer_id]

    result = [f"현재 포인트: {current_points:,}점\n"]

    if not history.empty:
        result.append("포인트 내역:")
        for _, record in history.iterrows():
            result.append(f"{record['일자']} {record['구분']} {record['포인트']:,}점")
            result.append(f"내용: {record['내용']}")
            result.append("-------------------")

    return "\n".join(result)

def get_current_promotions() -> str:
    """현재 진행중인 프로모션을 조회합니다."""
    today = datetime.now().strftime("%Y-%m-%d")
    active = promotions_df[
        (promotions_df['시작일'] <= today) &
        (promotions_df['종료일'] >= today)
    ]

    if active.empty:
        return "현재 진행중인 프로모션이 없습니다."

    result = ["현재 진행중인 프로모션:\n"]

    for _, promo in active.iterrows():
        result.append(f"[{promo['제목']}]")
        result.append(f"내용: {promo['내용']}")
        result.append(f"기간: {promo['시작일']} ~ {promo['종료일']}")
        result.append("-------------------")

    return "\n".join(result)

def get_popular_products(category: str = None, period: str = "month") -> str:
    """인기상품을 조회합니다."""
    df = products_df.copy()

    if category:
        df = df[df['카테고리'].str.contains(category, case=False, na=False)]

    # 판매량 기준 정렬
    df = df.nlargest(5, '판매량')

    if df.empty:
        return "조건에 맞는 상품이 없습니다."

    result = ["인기상품 TOP 5:\n"]

    for i, (_, product) in enumerate(df.iterrows(), 1):
        result.append(f"{i}. {product['상품명']}")
        result.append(f"   가격: {product['가격']:,}원")
        result.append(f"   판매량: {product['판매량']}개")
        result.append("-------------------")

    return "\n".join(result)

In [7]:
# 함수 실행 예시 - 실제 자연어 질문 시나리오
print("=== 1. 고객 프로필 조회 ===")
print("질문: '내 정보 좀 알려줘' (고객이 C001이라고 가정)")
print(get_customer_profile("C001"))
print("\n질문: '박예준 고객 정보 확인해줘'")
print(get_customer_profile("C001"))
print("\n")

print("=== 2. 상품 검색 ===")
print("질문: 'LED 제품 있어?'")
print(search_products(keyword="LED"))
print("\n질문: '간식 카테고리에서 3만원 이하 상품 보여줘'")
print(search_products(category="간식", price_max=30000))
print("\n질문: '스마트펫에서 나온 상품 뭐 있어?'")
print(search_products(keyword="스마트펫"))
print("\n")

print("=== 3. 고객 주문 내역 조회 ===")
print("질문: '내가 주문한 거 다 보여줘' (고객이 C001이라고 가정)")
print(get_customer_orders("C001"))
print("\n질문: 'C001 고객이 3월에 주문한 거 조회해줘'")
print(get_customer_orders("C001", start_date="2025-03-01", end_date="2025-03-31"))
print("\n질문: '김미숙 고객 주문 내역 확인해줘'")
print(get_customer_orders("C002"))
print("\n")

print("=== 4. 배송 상태 확인 ===")
print("질문: 'O0001 주문 배송 상태 알려줘'")
print(get_delivery_status(order_id="O0001"))
print("\n질문: '내 배송 현황 다 보여줘' (고객이 C001이라고 가정)")
print(get_delivery_status(customer_id="C001"))
print("\n질문: 'C002 고객 배송 상태 확인해줘'")
print(get_delivery_status(customer_id="C002"))
print("\n")

print("=== 5. 리뷰 검색 ===")
print("질문: '내가 쓴 리뷰 다 보여줘' (고객이 C001이라고 가정)")
print(search_reviews(customer_id="C001"))
print("\n질문: '자동급식기 리뷰 있어?'")
print(search_reviews(product_name="자동급식기"))
print("\n질문: '편리하다는 리뷰 찾아줘'")
print(search_reviews(keyword="편리"))
print("\n질문: '평점 5점인 리뷰만 보여줘'")
print(search_reviews(rating=5))
print("\n")

print("=== 6. 장바구니 조회 ===")
print("질문: '내 장바구니에 뭐 있어?' (고객이 C001이라고 가정)")
print(get_customer_cart("C001"))
print("\n질문: 'C002 고객 장바구니 확인해줘'")
print(get_customer_cart("C002"))
print("\n")

print("=== 7. 포인트 내역 조회 ===")
print("질문: '내 포인트 얼마야?' (고객이 C001이라고 가정)")
print(get_point_history("C001"))
print("\n질문: 'C002 고객 포인트 내역 보여줘'")
print(get_point_history("C002"))
print("\n")

print("=== 8. 현재 프로모션 조회 ===")
print("질문: '지금 무슨 이벤트 하고 있어?'")
print(get_current_promotions())
print("\n질문: '할인 행사 있나요?'")
print(get_current_promotions())
print("\n")

print("=== 9. 인기상품 조회 ===")
print("질문: '요즘 뭐가 잘 나가?'")
print(get_popular_products())
print("\n질문: '간식 카테고리에서 인기있는 거 추천해줘'")
print(get_popular_products(category="간식"))
print("\n질문: '베스트셀러 보여줘'")
print(get_popular_products())
print("\n")

=== 1. 고객 프로필 조회 ===
질문: '내 정보 좀 알려줘' (고객이 C001이라고 가정)
고객ID: C001
고객명: 박예준
전화번호: 010-1234-5678
이메일: park@example.com
가입일자: 2020-08-10
고객등급: VIP
보유포인트: 5,010점
총구매액: 892,540원
구매횟수: 12회

질문: '박예준 고객 정보 확인해줘'
고객ID: C001
고객명: 박예준
전화번호: 010-1234-5678
이메일: park@example.com
가입일자: 2020-08-10
고객등급: VIP
보유포인트: 5,010점
총구매액: 892,540원
구매횟수: 12회


=== 2. 상품 검색 ===
질문: 'LED 제품 있어?'
검색 결과: 1개 상품

상품ID: P006
상품명: LED 목줄
가격: 18,900원
재고: 75개
카테고리: 외출용품
-------------------

질문: '간식 카테고리에서 3만원 이하 상품 보여줘'
조건에 맞는 상품이 없습니다.

질문: '스마트펫에서 나온 상품 뭐 있어?'
검색 결과: 1개 상품

상품ID: P003
상품명: 자동 급식기
가격: 89,900원
재고: 10개
카테고리: 전자제품
-------------------


=== 3. 고객 주문 내역 조회 ===
질문: '내가 주문한 거 다 보여줘' (고객이 C001이라고 가정)
주문 내역 (4건):

주문ID: O0001
주문일자: 2025-03-25
상품: 프리미엄 강아지 간식 세트
수량: 2개
결제금액: 53,346원
결제상태: 결제완료
-------------------
주문ID: O0002
주문일자: 2025-03-20
상품: LED 목줄
수량: 1개
결제금액: 18,900원
결제상태: 결제완료
-------------------
주문ID: O0003
주문일자: 2025-02-28
상품: 자동 급식기
수량: 1개
결제금액: 89,900원
결제상태: 결제완료
-------------------
주문ID: O0004
주문일자: 202

In [8]:
tools = [
    FunctionTool.from_defaults(fn=f)
    for f in [
        get_customer_profile,
        search_products,
        get_customer_orders,
        get_delivery_status,
        search_reviews,
        get_customer_cart,
        get_point_history,
        get_current_promotions,
        get_popular_products,
    ]
]

In [ ]:
# LLM 설정
llm = OpenAI(model="gpt-4o", temperature=0, api_key='Your API KEY')

# 시스템 프롬프트
SYSTEM_PROMPT = """당신은 쇼핑몰 고객 지원 도우미입니다. 사용자의 질문에 최선을 다해 답변하세요.

1. 고객 프로필, 주문 내역, 배송 상태, 포인트, 결제 정보 등에 관한 질문은 반드시 도구를 호출하여 답변합니다.
2. 이전 대화 맥락을 참고하여 일관된 답변을 제공합니다.
3. 고객의 문의에 친절하고 전문적으로 응대합니다.
4. ID 형식을 정확히 사용하십시오.
5. 현재 로그인한 사용자의 ID는 C001로 가정합니다.
"""

In [10]:
# FunctionAgent 생성
agent = FunctionAgent(
    tools=tools,
    llm=llm,
    system_prompt=SYSTEM_PROMPT
)

In [15]:
class ConversationManager:
    def __init__(self):
        self.agent = agent
        self.tool_params = {}
        for tool in tools:
            if hasattr(tool, 'fn'):
                import inspect
                sig = inspect.signature(tool.fn)
                self.tool_params[tool.metadata.name] = list(sig.parameters.keys())

    async def aprocess_message(self, message: str):
        response = await self.agent.run(message)

        # response에서 텍스트 추출
        if hasattr(response, 'response'):
            raw_answer = response.response
        else:
            raw_answer = response

        # ChatMessage 객체면 content 또는 str()로 변환
        if hasattr(raw_answer, 'content'):
            answer = raw_answer.content
        else:
            answer = str(raw_answer)

        # assistant: 접두사 제거
        import re
        s = str(answer).lstrip()
        answer = re.sub(r'^(?:assistant)\s*:\s*', '', s, count=1, flags=re.IGNORECASE)

        log_content = [f"[User Input] {message}"]

        # tool_calls에서 Function Call 정보 추출
        if hasattr(response, 'tool_calls') and response.tool_calls:
            for tool_call in response.tool_calls:
                if hasattr(tool_call, 'tool_name'):
                    log_content.append(f"[Function Call] {tool_call.tool_name}")

                    # Parameters 정보 추출
                    if hasattr(tool_call, 'tool_kwargs') and tool_call.tool_kwargs:
                        params = []
                        for key, value in tool_call.tool_kwargs.items():
                            params.append(f"{key}: {value}")
                        log_content.append(f"[Parameters] {', '.join(params)}")
                    else:
                        log_content.append(f"[Parameters] (없음)")

                    # Output 정보 추출
                    if hasattr(tool_call, 'raw_output') and tool_call.raw_output:
                        log_content.append(f"[Output] {str(tool_call.raw_output)}")
                    elif hasattr(tool_call, 'tool_output') and hasattr(tool_call.tool_output, 'blocks'):
                        for block in tool_call.tool_output.blocks:
                            if hasattr(block, 'text'):
                                log_content.append(f"[Output] {str(block.text)}")
                                break
                    else:
                        log_content.append(f"[Output] (출력 정보 없음)")
        log_content.append(f"[AI Response] {answer}")  # ← 이 줄 추가
        execution_log = "\n".join(log_content)
        return answer, execution_log

    def clear_history(self):
        global agent
        agent = FunctionAgent(
            tools=tools,
            llm=llm,
            system_prompt=SYSTEM_PROMPT
        )
        self.agent = agent
        return []

In [16]:
conversation_manager = ConversationManager()

In [17]:
# 예제 1: 고객 프로필 조회
print("=== 예제 1: 고객 프로필 조회 ===")
query1 = "고객 ID C001의 프로필 정보를 알려주세요"
result1, log1 = await conversation_manager.aprocess_message(query1)
print("\n최종 답변:")
print(result1)
print("\n실행 로그:")
print(log1)

# 예제 2: 주문 내역 조회 (멀티턴 테스트)
print("\n=== 예제 2: 주문 내역 조회 (멀티턴) ===")
query2 = "그 고객의 주문 내역을 조회해주세요"
result2, log2 = await conversation_manager.aprocess_message(query2)
print("\n최종 답변:")
print(result2)
print("\n실행 로그:")
print(log2)

# 예제 3: 배송 상태 조회 (멀티턴 테스트)
print("\n=== 예제 3: 배송 상태 조회 (멀티턴) ===")
query3 = "배송 상태도 확인해주세요"
result3, log3 = await conversation_manager.aprocess_message(query3)
print("\n최종 답변:")
print(result3)
print("\n실행 로그:")
print(log3)

=== 예제 1: 고객 프로필 조회 ===

최종 답변:
고객 ID C001의 프로필 정보는 다음과 같습니다:

- 고객명: 박예준
- 전화번호: 010-1234-5678
- 이메일: park@example.com
- 가입일자: 2020-08-10
- 고객등급: VIP
- 보유포인트: 5,010점
- 총구매액: 892,540원
- 구매횟수: 12회

다른 도움이 필요하시면 언제든지 말씀해 주세요!

실행 로그:
[User Input] 고객 ID C001의 프로필 정보를 알려주세요
[Function Call] get_customer_profile
[Parameters] customer_id: C001
[Output] 고객ID: C001
고객명: 박예준
전화번호: 010-1234-5678
이메일: park@example.com
가입일자: 2020-08-10
고객등급: VIP
보유포인트: 5,010점
총구매액: 892,540원
구매횟수: 12회
[AI Response] 고객 ID C001의 프로필 정보는 다음과 같습니다:

- 고객명: 박예준
- 전화번호: 010-1234-5678
- 이메일: park@example.com
- 가입일자: 2020-08-10
- 고객등급: VIP
- 보유포인트: 5,010점
- 총구매액: 892,540원
- 구매횟수: 12회

다른 도움이 필요하시면 언제든지 말씀해 주세요!

=== 예제 2: 주문 내역 조회 (멀티턴) ===

최종 답변:
고객님의 주문 내역은 다음과 같습니다:

1. **주문ID: O0001**
   - 주문일자: 2025-03-25
   - 상품: 프리미엄 강아지 간식 세트
   - 수량: 2개
   - 결제금액: 53,346원
   - 결제상태: 결제완료

2. **주문ID: O0002**
   - 주문일자: 2025-03-20
   - 상품: LED 목줄
   - 수량: 1개
   - 결제금액: 18,900원
   - 결제상태: 결제완료

3. **주문ID: O0003**
   - 주문일자: 2025-02-2

In [ ]:
async def ui_process_message(message, history):
    if not message:
        return "", history, ""
    answer, log = await conversation_manager.aprocess_message(message)
    
    # Gradio 6.x 호환을 위한 딕셔너리 포맷 사용
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": answer})
    
    return "", history, log

def clear_conversation():
    conversation_manager.clear_history()
    return [], ""

with gr.Blocks(css="footer {visibility: hidden}") as demo:
    gr.Markdown("# 쇼핑몰 고객 지원 에이전트 (LlamaIndex Function Calling)")
    gr.Markdown("고객 프로필, 주문 내역, 배송 상태, 포인트, 결제 정보 등에 관한 질문을 해보세요.")

    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(height=500)
            msg = gr.Textbox(label="질문을 입력하세요", placeholder="예: 고객 ID C001의 프로필 정보를 알려주세요")
            clear = gr.Button("대화 초기화")
        with gr.Column(scale=1):
            log_output = gr.Textbox(label="실행 로그", lines=15)

    msg.submit(ui_process_message, inputs=[msg, chatbot], outputs=[msg, chatbot, log_output])
    clear.click(clear_conversation, outputs=[chatbot, log_output])

    gr.Markdown("""
    ### 예시 질문:
    - 고객 ID C001의 프로필 정보를 알려주세요
    - 그 고객의 주문 내역을 조회해주세요
    - 배송 상태도 확인해주세요
    - 고객 ID C002의 주문 내역을 조회해주세요
    - 고객 ID C003의 배송 상태를 확인하고 싶어요
    - 고객 ID C004의 포인트 정보가 궁금합니다
    """)

demo.launch(debug=True)

/tmp/ipython-input-140772758.py:18: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e99138ec9490ca7edd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
